In [67]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Base URL
base_url = "https://www.mpmmedicalsupply.com/"
driver.get(base_url)

wait = WebDriverWait(driver, 10)

# Open CSV file for saving data
with open("products.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Category", "Subcategory", "Product Title", "Price", "Product Link"])

    # Wait for the category items to load
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".menulist-item")))

    # Find all category items with the class name ".menulist-item"
    category_elements = driver.find_elements(By.CSS_SELECTOR, ".menulist-item")

    print(f"Found {len(category_elements)} categories")

    # Ensure we have at least 2 categories (since we want to target item 2)
    if len(category_elements) >= 2:
        # Get the second category (index 1 for the second item)
        category = category_elements[5]  # Target item 2 (index 1)

        try:
            # Click on the "View All" link inside the second category
            view_all_link = category.find_element(By.CSS_SELECTOR, ".menulist-menu__item--last a")
            view_all_href = view_all_link.get_attribute("href")
            category_name = category.text.strip()
            print(f"Navigating to: {view_all_href}")
            driver.get(view_all_href)

            # Wait for the subcategory menu to load
            subcategory_menu = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 
                ".subcollections-featured-collection__subcollection-menu-wrapper ul")))

            # Extract all subcategory links
            subcategory_links = subcategory_menu.find_elements(By.CSS_SELECTOR, "li > a")
            subcategory_urls = [(link.get_attribute("href"), link.text.strip()) for link in subcategory_links if link.get_attribute("href")]

            for sub_url, subcategory_name in subcategory_urls:
                print(f"Visiting subcategory: {sub_url}")
                driver.get(sub_url)

                # Wait for product grid to load
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".productgrid--items")))

                # Extract products
                products = driver.find_elements(By.CSS_SELECTOR, ".productitem--info")

                for product in products:
                    try:
                        title = product.find_element(By.CSS_SELECTOR, ".productitem--title").text.strip()
                        price = product.find_element(By.CSS_SELECTOR, ".price").text.strip()
                        product_link = product.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                        print(f"Product: {title} | Price: {price} | Link: {product_link}")

                        # Save to CSV
                        writer.writerow([category_name, subcategory_name, title, price, product_link])

                    except Exception as e:
                        print(f"Error extracting product details: {e}")

            # Go back to the main category page
            driver.get(base_url)
            time.sleep(2)

        except Exception as e:
            print(f"Error navigating category: {e}")

    else:
        print("There are not enough categories. Please check the page.")

# Close the driver
driver.quit()


Found 6 categories
Navigating to: https://www.mpmmedicalsupply.com/collections/surgical-instruments
Visiting subcategory: https://www.mpmmedicalsupply.com/collections/curettes
Product: Endometrial Biopsy Sampler (Box of 25) | Price: $185.00 | Link: https://www.mpmmedicalsupply.com/collections/curettes/products/endometrial-sampler
Product: Sims Uterine Curette, Sharp | Price: $65.00 | Link: https://www.mpmmedicalsupply.com/collections/curettes/products/sims-uterine-curette
Product: Sims (Thomas) Uterine Curette, Blunt | Price: $69.00 | Link: https://www.mpmmedicalsupply.com/collections/curettes/products/sims-thomas-uterine-curette-blunt
Product: Heaney Uterine Curette | Price: $86.00 | Link: https://www.mpmmedicalsupply.com/collections/curettes/products/heaney-uterine-curette
Product: Novak Endocervical Suction Biopsy Curette | Price: $40.00 | Link: https://www.mpmmedicalsupply.com/collections/curettes/products/novak-curette
Product: Kevorkian-Younge Endocervical Biopsy Curette | Price: